In [ ]:
# install PyRadiomics if not installed already
!pip install pyradiomics

     |████████████████████████████████| 188 kB 5.0 MB/s 
     |████████████████████████████████| 51.7 MB 144 kB/s 
     |████████████████████████████████| 109 kB 55.7 MB/s 
     |████████████████████████████████| 546 kB 45.3 MB/s 


In [ ]:
# importing libraries
from __future__ import print_function
import sys
import os
import collections
import SimpleITK as sitk
from SimpleITK.SimpleITK import JoinSeries
import logging
import six
import radiomics
from radiomics import featureextractor, getFeatureClasses, getTestCase, firstorder, glcm, imageoperations, shape, glrlm, glszm
import numpy as np
import pandas as pd
import seaborn as sns
import zipfile
import PIL
from PIL import Image
from google.colab import files
import pandas as pd
import csv

In [ ]:
!wget --no-check-certificate \
    "https://drive.google.com/uc?export=download&id=1qqAwSwjhRvlCelSwt8vqlkY4gvg_2yDc" \
    -O "/tmp/chest-x-rays.zip"

zip_ref = zipfile.ZipFile('/tmp/chest-x-rays.zip', 'r') # Opens the zip file in read mode
zip_ref.extractall('/tmp') # Extracts the files into the /tmp folder
zip_ref.close()

#zip_ref = zipfile.ZipFile('/tmp/covid_19_severity_prediction-Justin/Chest X-ray Image Data sets.zip', 'r') # Opens the zip file in read mode
#zip_ref.extractall('/tmp') # Extracts the files into the /tmp folder
#zip_ref.close()


--2022-04-19 00:36:23--  https://drive.google.com/uc?export=download&id=1qqAwSwjhRvlCelSwt8vqlkY4gvg_2yDc
Resolving drive.google.com (drive.google.com)... 108.177.11.102, 108.177.11.138, 108.177.11.139, ...
Connecting to drive.google.com (drive.google.com)|108.177.11.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-3k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ajrn5or4stbk05jfcdn3v3j2t4asbjr1/1650328575000/10701616149928504389/*/1qqAwSwjhRvlCelSwt8vqlkY4gvg_2yDc?e=download [following]
--2022-04-19 00:36:24--  https://doc-14-3k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ajrn5or4stbk05jfcdn3v3j2t4asbjr1/1650328575000/10701616149928504389/*/1qqAwSwjhRvlCelSwt8vqlkY4gvg_2yDc?e=download
Resolving doc-14-3k-docs.googleusercontent.com (doc-14-3k-docs.googleusercontent.com)... 173.194.211.132, 2607:f8b0:400c:c10::84
Connecting to doc-14-3k-docs.googleusercontent.com (doc-14-3k-doc

In [ ]:
# First define the settings of extractor
settings = {}
settings['label'] = 255
#settings['binWidth'] = 25
#settings['resampledPixelSpacing'] = None
#settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
#settings['interpolator'] = 'sitkBSpline'
#settings['verbose'] = True
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
print('Extraction parameters:\n\t', extractor.settings)

# Enable a filter (in addition to the 'Original' filter already enabled)
extractor.enableImageTypeByName('LoG')
print('')
print('Enabled filters:\n\t', extractor.enabledImagetypes)

# Specify feature classes, all are enabled by default
extractor.disableAllFeatures()
extractor.enableAllFeatures()
#extractor.enableFeatureClassByName('firstorder')

# Specify some additional features in the GLCM feature class
#extractor.enableFeaturesByName(glcm=['Autocorrelation', 'Homogeneity1', 'SumSquares'])
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True}

Enabled filters:
	 {'Original': {}, 'LoG': {}}

Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': [], 'shape2D': []}


In [ ]:
dataDir = os.path.join('/tmp/results2')
print("dataDir, relative path:", dataDir)
print("dataDir, absolute path:", os.path.abspath(dataDir))

data = []

count = 1
for file in os.listdir(dataDir):
  if(count > len(os.listdir(dataDir))/2):
    break
  # Define the testcase name
  CXR_name = str(count)
  # Store the file paths of our testing image and label map into two variables
  imagePath = os.path.join(dataDir, CXR_name + ".jpg")
  maskPath = os.path.join(dataDir, CXR_name + "-mask.png")
  # Transforming into "3D" image for pyradiomics
  image = sitk.ReadImage(imagePath, sitk.sitkInt16)
  image3d = sitk.JoinSeries(image)
  mask = sitk.ReadImage(maskPath, sitk.sitkInt16)
  mask3d = sitk.JoinSeries(mask)

  # Extract features
  result = extractor.execute(image3d, mask3d)
  # print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
  # print('Features extracted for image ' + str(count))
  # print('Result type:', type(result))  # result is returned in a Python ordered dictionary
  # print('')
  # print('Calculated features')
  feature_value = []
  header = []
  for key, value in six.iteritems(result):
      header.append(key)
      feature_value.append(value)
  data.append(feature_value[22:])
  count = count + 1

with open("frequencies2.csv", "w") as outfile:
  csvwriter = csv.writer(outfile)
  csvwriter.writerow(header[22:])
  csvwriter.writerows(data)


dataDir, relative path: /tmp/results2
dataDir, absolute path: /tmp/results2


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Image too small to apply LoG filter, size: [256 256   1]
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Image too small to apply LoG filter, size: [256 256   1]
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Image too small to apply LoG filter, size: [256 256   1]
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Image too small to apply LoG filter, size: [256 256   1]
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Averag